In [1]:
from ortools.linear_solver import pywraplp

In [2]:
import pickle
import numpy as np

# Loading the list from the pickle file
file_path = r"F:\dist_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    dist_list = pickle.load(file)

print("Dist list loaded successfully.")
print(np.shape(dist_list))

Dist list loaded successfully.
(400, 10450)


In [3]:
dist_list =  np.transpose(dist_list, (1, 0))

In [15]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [60, 120, 50, 100],
    [50, 100, 90, 400],
    [50, 200, 170, 120],
    [50, 200, 190, 160]
]
num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i} assigned to task {j}." + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")


Total cost = 240.0

Worker 0 assigned to task 1. Cost: 80
Worker 1 assigned to task 3. Cost: 65
Worker 3 assigned to task 0. Cost: 45
Worker 4 assigned to task 2. Cost: 50


In [4]:
from tqdm import tqdm

costs = dist_list

num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in tqdm(range(num_workers)):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

sol_indexes = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.1:
                print(f"Cluster {j} assigned to Class {i}." + f" Cost: {costs[i][j]}")
                sol_indexes.append(i)
else:
    print("No solution found.")

100%|████████████████████████████████████████████████████████████████████████████| 10450/10450 [01:58<00:00, 88.36it/s]


Total cost = 1709.9074519929786

Cluster 267 assigned to Class 57. Cost: 2.1476203811925036
Cluster 278 assigned to Class 58. Cost: 1.044421980393585
Cluster 369 assigned to Class 69. Cost: 2.53339415416598
Cluster 330 assigned to Class 82. Cost: 1.3207315018024208
Cluster 92 assigned to Class 90. Cost: 4.971960963614804
Cluster 145 assigned to Class 94. Cost: 2.3658444737646853
Cluster 63 assigned to Class 95. Cost: 5.1959978279351695
Cluster 291 assigned to Class 96. Cost: 1.2960433747270226
Cluster 1 assigned to Class 103. Cost: 7.309396832851805
Cluster 148 assigned to Class 111. Cost: 4.4450791836802654
Cluster 39 assigned to Class 137. Cost: 3.048945420863002
Cluster 255 assigned to Class 192. Cost: 3.101599565110415
Cluster 177 assigned to Class 238. Cost: 1.192256009024106
Cluster 288 assigned to Class 239. Cost: 1.3800359499975587
Cluster 394 assigned to Class 252. Cost: 0.8798381759394737
Cluster 167 assigned to Class 255. Cost: 1.8174502115853806
Cluster 324 assigned to Clas

Cluster 129 assigned to Class 4094. Cost: 2.493499685439575
Cluster 80 assigned to Class 4103. Cost: 2.291892591267221
Cluster 87 assigned to Class 4142. Cost: 1.9247790988426103
Cluster 186 assigned to Class 4180. Cost: 1.4541892027991246
Cluster 381 assigned to Class 4185. Cost: 3.8103346175179817
Cluster 192 assigned to Class 4187. Cost: 4.20737356841911
Cluster 160 assigned to Class 4198. Cost: 1.342084645109644
Cluster 24 assigned to Class 4202. Cost: 2.1450966623752405
Cluster 373 assigned to Class 4216. Cost: 2.979508646864761
Cluster 139 assigned to Class 4225. Cost: 2.209920276085398
Cluster 201 assigned to Class 4249. Cost: 1.1719685085982854
Cluster 79 assigned to Class 4253. Cost: 3.3628605002483662
Cluster 28 assigned to Class 4257. Cost: 1.7716339360208253
Cluster 52 assigned to Class 4360. Cost: 1.297449275818937
Cluster 172 assigned to Class 4385. Cost: 1.6359714314577438
Cluster 292 assigned to Class 4446. Cost: 5.416786147820151
Cluster 193 assigned to Class 4447. Cos

Cluster 111 assigned to Class 8189. Cost: 1.6206230550831746
Cluster 234 assigned to Class 8198. Cost: 4.765922525313198
Cluster 365 assigned to Class 8202. Cost: 6.21411134735423
Cluster 384 assigned to Class 8249. Cost: 3.206241776793936
Cluster 0 assigned to Class 8253. Cost: 4.793997596828856
Cluster 32 assigned to Class 8447. Cost: 18.4250794208443
Cluster 60 assigned to Class 8450. Cost: 10.388161790808915
Cluster 64 assigned to Class 8497. Cost: 12.943127408904369
Cluster 168 assigned to Class 8529. Cost: 6.411388415752522
Cluster 204 assigned to Class 8530. Cost: 10.63638515439814
Cluster 237 assigned to Class 8975. Cost: 2.119114628588786
Cluster 187 assigned to Class 8998. Cost: 1.6477550515394994
Cluster 191 assigned to Class 9011. Cost: 8.417745269444415
Cluster 86 assigned to Class 9012. Cost: 1.2760102092623784
Cluster 238 assigned to Class 9040. Cost: 3.1254163752796784
Cluster 125 assigned to Class 9059. Cost: 4.087865621717273
Cluster 231 assigned to Class 9067. Cost: 

In [5]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"F:\index_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(sol_indexes, file)

print("Combo list saved successfully.")


Combo list saved successfully.
